In [1]:
import pandas as pd
import yaml
import csv
from itertools import combinations

In [4]:
def load_config(config_path):
    with open(config_path, 'r') as file:
        config = yaml.safe_load(file)
    return config

config = load_config('config.yaml')

In [5]:
csv_location = config['data_paths']['output_file_full_dataframe']
df = pd.read_csv(csv_location)


In [6]:
total_number_of_matches = len(df['match_id'].unique())
average_number_of_events_per_match = len(df)/ total_number_of_matches
average_number_of_events_per_match

1680.9322033898304

Occurences of primary types:

In [7]:
unique_values_primary_type = df['primary_type'].value_counts()
unique_values_primary_type

primary_type
pass                 246263
duel                 118374
touch                 38542
interception          21952
game_interruption     19496
throw_in              11307
free_kick              7886
infraction             7387
shot                   6588
goal_kick              4612
clearance              3071
acceleration           2955
corner                 2875
shot_against           2482
offside                 906
goalkeeper_exit         705
fairplay                347
penalty                 106
own_goal                 21
Name: count, dtype: int64

First all uniques secondary types. Then the occurences of primary types in combination with the seperated secondary types

In [9]:
unique_secondary_types_separated = pd.Series([
    word.strip()
    for sublist in df['secondary_types'].dropna().str.split(',')
    for word in sublist
]).unique()
print(unique_secondary_types_separated)
print(len(unique_secondary_types_separated))

['back_pass' 'short_or_medium_pass' 'lateral_pass' 'forward_pass'
 'progressive_pass' 'loose_ball_duel' 'carry' 'long_pass'
 'pass_to_final_third' 'opportunity' 'head_pass' 'recovery'
 'under_pressure' 'ball_out' 'loss' 'hand_pass' 'foul_suffered'
 'ground_duel' 'linkup_play' 'offensive_duel' 'defensive_duel' 'foul'
 'dribble' 'aerial_duel' 'counterpressing_recovery' 'pass'
 'deep_completion' 'touch_in_box' 'interception' 'sliding_tackle'
 'progressive_run' 'smart_pass' 'cross' 'deep_completed_cross'
 'pass_to_penalty_area' 'second_assist' 'assist' 'key_pass' 'shot_assist'
 'goal' 'conceded_goal' 'dribbled_past_attempt' 'shot_after_throw_in'
 'shot_block' 'free_kick_cross' 'save' 'save_with_reflex'
 'shot_after_corner' 'yellow_card' 'cross_blocked' 'through_pass'
 'free_kick_shot' 'shot' 'head_shot' 'whistle' 'shot_after_free_kick'
 'acceleration' 'penalty_foul' 'penalty_goal' 'penalty_conceded_goal'
 'third_assist' 'red_card' 'penalty_save']
63


In [10]:
pd.options.display.max_rows = 2800

In [12]:
# Initialize an empty list to store tuples of (primary_type, secondary_type)
expanded_rows = []

# Iterate over each row in the DataFrame
for index, row in df.dropna(subset=['secondary_types']).iterrows():
    primary = row['primary_type']
    secondary_list = row['secondary_types'].split(',')
    for secondary in secondary_list:
        expanded_rows.append((primary, secondary.strip()))

# Create a new DataFrame from the expanded rows
expanded_df = pd.DataFrame(expanded_rows, columns=['primary_type', 'secondary_type'])

# Get the counts of secondary types per primary type
counts_per_primary_type = expanded_df.groupby('primary_type')['secondary_type'].value_counts()

# Calculate the percentages
total_counts_per_primary = expanded_df.groupby('primary_type')['secondary_type'].count()
percentages = (counts_per_primary_type / total_counts_per_primary).reset_index(name='percentage')

# Combine counts and percentages
counts_per_primary_type = counts_per_primary_type.reset_index(name='count')
result = pd.merge(counts_per_primary_type, percentages, on=['primary_type', 'secondary_type'])

In [13]:
result

,primary_type,secondary_type,count,percentage
0,acceleration,carry,2047,0.533351
1,acceleration,progressive_run,1744,0.454403
2,acceleration,linkup_play,45,0.011725
3,acceleration,third_assist,2,0.000521
4,clearance,loss,973,0.458530
5,clearance,head_pass,588,0.277097
6,clearance,under_pressure,341,0.160697
7,clearance,recovery,146,0.068803
8,clearance,counterpressing_recovery,54,0.025448
9,clearance,carry,19,0.008954


In [15]:
# Calculate conditional percentages
conditional_percentages = []

for primary_type in expanded_df['primary_type'].unique():
    subset_df = expanded_df[expanded_df['primary_type'] == primary_type]
    secondary_types = subset_df['secondary_type'].unique()
    
    for pair in combinations(secondary_types, 2):
        s1, s2 = pair
        s1_count = subset_df[subset_df['secondary_type'] == s1].shape[0]
        s2_count = subset_df[subset_df['secondary_type'] == s2].shape[0]
        common_count = subset_df[(subset_df['secondary_type'] == s1) & (subset_df['secondary_type'] == s2)].shape[0]
        
        if s1_count > 0:
            percentage = common_count / s1_count
            conditional_percentages.append((primary_type, s1, s2, percentage))
        
        if s2_count > 0 and s1 != s2:
            percentage = common_count / s2_count
            conditional_percentages.append((primary_type, s2, s1, percentage))

# Create DataFrame for conditional percentages
conditional_df = pd.DataFrame(conditional_percentages, columns=['primary_type', 'secondary_type1', 'secondary_type2', 'percentage'])

In [18]:
conditional_df

,primary_type,secondary_type1,secondary_type2,percentage
0,pass,back_pass,short_or_medium_pass,0.0
1,pass,short_or_medium_pass,back_pass,0.0
2,pass,back_pass,lateral_pass,0.0
3,pass,lateral_pass,back_pass,0.0
4,pass,back_pass,forward_pass,0.0
5,pass,forward_pass,back_pass,0.0
6,pass,back_pass,progressive_pass,0.0
7,pass,progressive_pass,back_pass,0.0
8,pass,back_pass,long_pass,0.0
9,pass,long_pass,back_pass,0.0


In [15]:
# Initialize a defaultdict to store counts
primary_secondary_count = defaultdict(lambda: defaultdict(int))

# Iterate through each row in the DataFrame
for index, row in df.iterrows():
    primary_type = row['primary_type']
    secondary_types = row['secondary_types']
    
    # Check if secondary_types is not NaN
    if pd.notna(secondary_types):
        secondary_types = secondary_types.split(',')  # Split by comma if necessary
        
        # Count occurrences of each secondary type for the current primary type
        for secondary_type in secondary_types:
            primary_secondary_count[primary_type][secondary_type.strip()] += 1  # Strip to remove any extra whitespace

# Convert defaultdict to regular dictionary for easier manipulation (if needed)
primary_secondary_count = dict(primary_secondary_count)

# Print the result
for primary_type, secondary_counts in primary_secondary_count.items():
    for secondary_type, count in secondary_counts.items():
        print(f"Primary Type: {primary_type}, Secondary Type: {secondary_type}, Occurrences: {count}")

Primary Type: pass, Secondary Type: back_pass, Occurrences: 36345
Primary Type: pass, Secondary Type: short_or_medium_pass, Occurrences: 213353
Primary Type: pass, Secondary Type: lateral_pass, Occurrences: 94132
Primary Type: pass, Secondary Type: forward_pass, Occurrences: 78835
Primary Type: pass, Secondary Type: progressive_pass, Occurrences: 37744
Primary Type: pass, Secondary Type: long_pass, Occurrences: 24686
Primary Type: pass, Secondary Type: pass_to_final_third, Occurrences: 27661
Primary Type: pass, Secondary Type: hand_pass, Occurrences: 1672
Primary Type: pass, Secondary Type: recovery, Occurrences: 16784
Primary Type: pass, Secondary Type: under_pressure, Occurrences: 11959
Primary Type: pass, Secondary Type: loss, Occurrences: 28135
Primary Type: pass, Secondary Type: head_pass, Occurrences: 9893
Primary Type: pass, Secondary Type: counterpressing_recovery, Occurrences: 8051
Primary Type: pass, Secondary Type: linkup_play, Occurrences: 6370
Primary Type: pass, Secondary

In [25]:
# Initialize an empty list to store tuples of (primary_type, secondary_type)
expanded_rows = []

# Iterate over each row in the DataFrame
for index, row in df.drop_duplicates(subset=['primary_type', 'secondary_types']).iterrows():
    primary = row['primary_type']
    secondary_types = row['secondary_types']
    
    # Check if secondary_types is NaN
    if pd.notna(secondary_types):
        secondary_list = secondary_types.split(',')
        for secondary in secondary_list:
            expanded_rows.append((primary, secondary.strip()))
    else:
        expanded_rows.append((primary, None))  # Handle NaN case

# Create a new DataFrame from the expanded rows
expanded_df = pd.DataFrame(expanded_rows, columns=['primary_type', 'secondary_type'])

# Get the counts of secondary types per primary type
counts_per_primary_type = expanded_df.groupby('primary_type')['secondary_type'].value_counts().reset_index(name='count')

# Calculate total counts per primary type for each secondary type
total_counts_per_primary_type = counts_per_primary_type.groupby('primary_type')['count'].sum().reset_index(name='total_count')

# Merge to get the total counts per primary type
counts_per_primary_type = pd.merge(counts_per_primary_type, total_counts_per_primary_type, on='primary_type')

# Calculate percentages
counts_per_primary_type['percentage'] = counts_per_primary_type['count'] / counts_per_primary_type['total_count']

# Initialize list to store conditional percentages
conditional_percentages = []

# Iterate over each primary type
for primary_type in expanded_df['primary_type'].unique():
    subset_df = expanded_df[expanded_df['primary_type'] == primary_type]
    secondary_types = subset_df['secondary_type'].unique()
    
    # Calculate total occurrences of each secondary type in this primary type
    total_counts = subset_df['secondary_type'].value_counts().reset_index(name='total_count')
    total_counts.rename(columns={'index': 'secondary_type', 'total_count': 'total_count_secondary_type'}, inplace=True)
    
    # Iterate over combinations of secondary types
    for s1, s2 in combinations(secondary_types, 2):
        s1_count = subset_df[subset_df['secondary_type'] == s1].shape[0]
        s2_count = subset_df[subset_df['secondary_type'] == s2].shape[0]
        common_count = subset_df[(subset_df['secondary_type'] == s1) & (subset_df['secondary_type'] == s2)].shape[0]
        
        # Calculate conditional percentage
        if s1_count > 0:
            total_count_s1 = total_counts.loc[total_counts['secondary_type'] == s1, 'total_count_secondary_type'].values[0]
            percentage = common_count / total_count_s1
            conditional_percentages.append((primary_type, s1, s2, percentage))
        
        if s2_count > 0 and s1 != s2:
            total_count_s2 = total_counts.loc[total_counts['secondary_type'] == s2, 'total_count_secondary_type'].values[0]
            percentage = common_count / total_count_s2
            conditional_percentages.append((primary_type, s2, s1, percentage))

# Create DataFrame for conditional percentages
conditional_df = pd.DataFrame(conditional_percentages, columns=['primary_type', 'secondary_type1', 'secondary_type2', 'conditional_percentage'])

# Merge with the main result DataFrame
final_result = pd.merge(counts_per_primary_type, conditional_df, left_on=['primary_type', 'secondary_type'], right_on=['primary_type', 'secondary_type1'])

# Drop unnecessary columns and sort
final_result = final_result[['primary_type', 'secondary_type', 'count', 'percentage', 'secondary_type2', 'conditional_percentage']]
final_result.sort_values(by=['primary_type', 'secondary_type', 'secondary_type2'], inplace=True)

print(final_result)

      primary_type  secondary_type  count  percentage  secondary_type2  \
2     acceleration           carry      5    0.384615      linkup_play   
0     acceleration           carry      5    0.384615  progressive_run   
3     acceleration           carry      5    0.384615     third_assist   
1     acceleration           carry      5    0.384615             None   
5     acceleration     linkup_play      3    0.230769            carry   
...            ...             ...    ...         ...              ...   
2777         touch  under_pressure      6    0.120000  progressive_run   
2780         touch  under_pressure      6    0.120000    second_assist   
2781         touch  under_pressure      6    0.120000     third_assist   
2775         touch  under_pressure      6    0.120000     touch_in_box   
2776         touch  under_pressure      6    0.120000             None   

      conditional_percentage  
2                        0.0  
0                        0.0  
3                 